# Quick example of implementing a simple payments network

There are 4 steps to running Veriphi guardrails on a tool given to an autonomous Agent.
 - Define network classes
 - Prepare data
 - Distribute and encrypt
 - Reconstruct and recover 


In [1]:
import { randomBytes } from 'crypto'
import * as ic from '@veriphi/veriphi_core';
import { encryptAESCTR, decryptAESCTR } from '@veriphi/veriphi_core/utils'; 

## Define network classes

In [2]:
// Run a network with 4 classes
const networkClasses: Record<string,number> = {
    'Agent': 0,
    'Authoriser': 1,
    'Endpoint': 2,
}

## Prepare data

In [3]:
type Wallet = {
  walletId: string;
  owner: {
    name: string;
    id: string;
  };
  defaultCurrency: string;
  addresses: Record<string, string>; // currency symbol -> address
};

// Assign to a variable
const wallet: Wallet = {
  walletId: "abc123",
  owner: {
    name: "Jane Doe",
    id: "user-6789"
  },
  defaultCurrency: "USD",
  addresses: {
    BTC: "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa",
    ETH: "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7"
  }
};

// Serialize to JSON string
const walletJson: string = JSON.stringify(wallet);
const encoder = new TextEncoder();
const walletBytes = encoder.encode(walletJson);

In [4]:
// Generate a public key afor distribution and a private key for obfuscation
const setupNode = new ic.SetupNode('node_setup') 
const seed = Buffer.from(randomBytes(32))
const publicKey = setupNode.genPublicKey(seed)
const privateKey = setupNode.genPrivateKey('obfPrivateKey', seed)
let [encrypted,nonce] = setupNode.encryptData(Buffer.from(walletBytes), privateKey);
// optionally set conditions with a minimum spend of 0 and a maximum spend of 1000
const [lowVal, highVal] = setupNode.implementConditions(0, 1000, privateKey);
let chunkSize;
[encrypted,chunkSize] = setupNode.obfuscateData(encrypted, privateKey, lowVal,highVal,50); // Dummy test value of 50 that should pass

[
  <Buffer c9 54 d0 b1 c1 93 7c 49 0c e4 73 7a 71 31 30 38 3d 57 21 bf 9e e1 53 1c 72 39 7a fb c0 01 66 77 e9 ff 00 af e1 29 80 3d c4 b8 5a 40 fe a2 29 fa 30 f1 ... 148 more bytes>,
  6
]


### The cell above can be wrapped up in 1 function call

In [5]:
const [publicData, privateData] = ic.setupNode(walletBytes, 0, 1000, true);

In [6]:
// Pack the data for distribution, with a specified identity for each class
let agentPacket = setupNode.packageData(encrypted, publicKey, 'E', networkClasses["Agent"])
let authPacket  = setupNode.packageData(encrypted, publicKey, 'E', networkClasses["Authoriser"])
let endpointPacket = setupNode.packageData(encrypted, publicKey, 'E', networkClasses["Endpoint"])

### Or more generally for many parties

In [7]:

let [agentPacket, authPacket, endpointPacket] = ic.distributeData(publicData, 'E', 3);

## Distribute and encrypt

### Example code for the Agent members

In [8]:
const agentNode = new ic.EncryptNode('agent_member')
const [publicKey, packetData, mode, identity ] = agentNode.unpackageData(agentPacket);
const agentPrivateKey = agentNode.genPrivateKey('privateKey', randomBytes(32));
const agentEncrypted = agentNode.encryptData(Buffer.from(packetData), agentPrivateKey, publicKey, mode, identity);
const agentEncPacket = agentNode.packageData(agentEncrypted, mode, identity);


### Or in simple function calls

In [9]:
const agentEncPacket    = ic.encryptNode(agentPacket, 'agent_member');
const authEncPacket     = ic.encryptNode(authPacket, 'authoriser_member');
const endpointEncPacket = ic.encryptNode(endpointPacket, 'endpoint_member');

## Reconstruction and recovery is conditioned on complete visibility and correct conditions

In [10]:
const veriphier = new ic.DecryptNode('Veriphier');
const partyData = veriphier.collectPackets(agentEncPacket, authEncPacket, endpointEncPacket);
const partyDataRecov = veriphier.recoverPackets(partyData);
const streamList = veriphier.reconstructData(partyDataRecov);
const reconstructed = veriphier.reassembleData(streamList, partyData[0].mode);
// attempt recovery with a spend of 500, which should pass checks
const [recovered,chunkSize] = veriphier.obfuscateData(reconstructed, privateData.key, privateData.low_val, privateData.high_val, 500);
const decrypted = veriphier.decryptData(recovered, privateData.nonce, privateData.key);


### Again, the above logic is wrapped up in a sigle call

In [11]:
const decrypted = ic.decryptNode(privateData, 500, true, agentEncPacket, authEncPacket, endpointEncPacket);

## The original wallet is recovered

In [12]:
const decoder = new TextDecoder
const recoveredWallet = decoder.decode(decrypted)

In [13]:
// original wallet
console.log("Original wallet is  ", wallet)

Original wallet is   {
  walletId: 'abc123',
  owner: { name: 'Jane Doe', id: 'user-6789' },
  defaultCurrency: 'USD',
  addresses: {
    BTC: '1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa',
    ETH: '0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7'
  }
}


In [14]:
// recovered wallet
console.log("Recovered wallet is ", JSON.parse(recoveredWallet))


Recovered wallet is  {
  walletId: 'abc123',
  owner: { name: 'Jane Doe', id: 'user-6789' },
  defaultCurrency: 'USD',
  addresses: {
    BTC: '1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa',
    ETH: '0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7'
  }
}


# Interfere with the scheme (change private keys, arguments, public key between parties) to break the decryption

### For example, decrypting with a spend larger then 1000 (the minimum spend)

In [15]:
const decrypted = ic.decryptNode(privateData, 1500, true, agentEncPacket, authEncPacket, endpointEncPacket);
const decoder = new TextDecoder
const recoveredWallet = decoder.decode(decrypted)
console.log("Recovered wallet is ", JSON.parse(recoveredWallet))


<anonymous_script>:1
i�%��y?��"��"abc12is��Yj�-F�떂ˈ��G��;����Z��VHջ2`4<)M��
^

SyntaxError: Unexpected token 'i', "i�%��y?��""... is not valid JSON
    at JSON.parse (<anonymous>)
    at evalmachine.<anonymous>:9:63
    at evalmachine.<anonymous>:11:3
    at sigintHandlersWrap (node:vm:280:12)
    at Script.runInThisContext (node:vm:135:14)
    at Object.runInThisContext (node:vm:317:38)
    at Object.execute (/Users/petervincent/Documents/veriphi/node/veriphi_core/node_modules/tslab/dist/executor.js:160:38)
    at JupyterHandlerImpl.handleExecuteImpl (/Users/petervincent/Documents/veriphi/node/veriphi_core/node_modules/tslab/dist/jupyter.js:250:38)
    at /Users/petervincent/Documents/veriphi/node/veriphi_core/node_modules/tslab/dist/jupyter.js:208:57
    at async JupyterHandlerImpl.handleExecute (/Users/petervincent/Documents/veriphi/node/veriphi_core/node_modules/tslab/dist/jupyter.js:208:21)
